# Cartesian Tree with implicit keys

In previous lecture we spoke about Search tree and it implementation calling Cartesian tree. However we missed one important part - functions on segments. For example to calculate the sum of array segment you need to go through all segment and sum up elements inside it. But with Cartesian Tree we can made some precalculation that can help answer to request faster (about O(logN)). But previously lets speak about split and merge of arrays.

## Implicit keys

On more time - in previous lecture we mentioned that we can split and merge arrays with log(N) complexity. But to do merge of two array-trees we need increase all keys inside the tree that will go to 'right' parts. But it should take from us about O(N) steps that is to much. To solve this problem lets just remember that our keys is indexes of some array and will not right them in nodes of array. With such rule there is no more problem with increasing keys. But how we now will work with the keys, when we need find some new element inside array?

![cartesianTree2](images/cartesianTree2.png)

To solve this problem we need keep something inside array. Something else than indices. Something that can help us calculate the key and will modified inside merge and split operations. Lets remember what index is exactly mean. It is mean the amount of elements that stays before our element in array. So, if element have index\key equal 5 that mean that there are 5 elements lies before this one. Lets image that this element is the root of the tree. The key equal 5 will exactly mean that there 5 elements inside left subtree. In other world the index of the root is equal to amount of elements in the left subtree.

Now lets move to the right. The index\key of this element is bigger than all elements in left subtree and all elements in all left subtrees that lies above. But it lower than all indices in right subtree by definition of search tree. Lets remember than all keys inside usual array (and cartesian tree that implement such array) lies one by one, mean two adjacent keys do not differ by more than one. And this mean that our key is exactly the amount of elements that lies in all left subtrees above current element plus left subtree of current element. So, in sum up, we can calculate the amount of elements inside each left subtree with help of the keys... or we can calculate each key by the amount of element in left subtrees. Plus to this we can recalculate the amount of element in subtree in merge and split operations. So, this is our solution.

Now instead of keys we will keep inside nodes the amount of elements inside of subtree. To calculate the key of current element we need just sum up all amounts of elements of each left subtree that we meet till this element.

![cartesian3](images/cartesianTree3.png)

In [3]:
class cartesianNode:
    data = None
    left = None
    right = None
    subtreeSum = 1

    def __recurSearch (self, key, sumAbove = 0):
        if (left == None): currentKey = sumAbove
        else: currentKey = sumAbove + left.subtreeSum
        if (currentKey == key): return self
        if (currentKey > key): return left.__recurSearch (key, sumAbove)
        if (currentKey < key): return right.__recurSearch (key, currentKey + 1)

    def search (self, key):
        return __recurSearch (key, 0)

### Merge
Ok, now we need to modify functions merge and split to work with implicit keys. Firstly lets remember how the work.

Merge take two trees as input data and return one merged. One important thing is that the 'left' tree should contain keys less than in 'right' tree. But now we have not any keys, just the amounts of elements is subtrees. So, lets just image that 'right' tree already have all keys granter than 'left'.

As you remember merge work recursively. At each step we need take the merge of left tree and left subtree of right tree or right tree and right subtree of left tree. After that we create a new tree with the root as element with the biggest value from left and right tree's root. And if we suggest that all sub trees already have correct amount of elements inside them that to calculate a new amount we need just sum them with the root '1'. And that is all, we have a new merge function.

In [4]:
def merge (tree1, tree2):
    if tree1 == None: return tree2
    if tree2 == None: return tree1
    
    resultTree = cartesianNode()
    if (tree1.data >= tree2.data):
        resultTree.right = merge (tree1.right, tree2)
        resultTree.left = tree1.left
        resultTree.subtreeSum = resultTree.right + resultTree.left + 1

    if (tree2.data > tree1.data):
        resultTree.left = merge (tree1, tree2.left)
        resultTree.right = tree2.right

    resultTree.subtreeSum = resultTree.right if resultTree.right != None else 0 + resultTree.left if resultTree.left != None else 0 + 1
    
    return resultTree

### Split

Same as for merge we need remember how it work and where we can insert sum update. So, the split take as input one tree and key and return two trees, divided by key. 

- Firstly we need to know the key which means we should calculate it during recursive calls.
- Secondly we need to calculate a new sum for two trees.

Ok, calculate the key is simple. We just use the same mechanism as for search function. Now let's talk about the resulting trees. One of the result trees is left or right subtree of current element. And it already has correct amount of elements. Other two trees that we take from recursion has correct amounts too (recursion guess). So we need just recalculate amount of elements for one of the resulting trees.

For example we have left subtree, new one 'middle' subtree and new one 'right' subtree that we take from division of right subtree. In result we need return 'merge' of left and middle subtrees and right subtree as second result. So, right subtree already has the correct amount of elements. So we need just recalculate it for new one left subtree (by sum amount of elements from left and middle trees) and that is all.

![splitCarTree](images/cartesianSplit.png)

In [6]:
def splitRec (tree, k, sumAbove = 0):
    if tree == None:
        return (None, None)
    
    if (left == None): currentKey = sumAbove
    else: currentKey = sumAbove + left.subtreeSum

    if (tree.key <= k):
        leftTree, rightTree = splitRec (tree.right, k, currentKey + 1)
        tree.right = leftTree
        tree.subtreeSum = tree.right.subTreeSum if tree.right != None else 0 + tree.left.subTreeSum if tree.left != None else 0 + 1
        return (tree, rightTree)
        
    if (tree.key > k):
        leftTree, rightTree = splitRec (tree.left, k, sumAbove)
        tree.left = rightTree
        tree.subtreeSum = tree.right.subTreeSum if tree.right != None else 0 + tree.left.subTreeSum if tree.left != None else 0 + 1
        return (leftTree, tree)

def split (tree, k):
    return splitRec(tree, k, 0)

## Functions on segments

So, now we can split and merge arrays with speed about O(logN). But what about other functions. Above we already discuss how to calculate amount of elements in subtree. But can we add some other functions to calculation process? Lets take a look.

We already have the amount of elements. Lets try change it to some other function, for example sum of values of elements. Ok, we know sum of elements in left tree, know it in right tree. And we can use the same process of recalculation as for amount of elements by changed the 1 in sum to value of element.

Now we have the sum of elements in every subtree. Lets try calculate sum of element in some segment. For example \[a;b\]. So, we need find the tree that contain all elements from a to b. But why we need to search them? Lets just split the tree into 3 parts. Left one will contain all elements from `0` to `a`, middle will contain from `a` to `b` and right one from `b` to the `end` of array. It can be done by two `splits`. Our answer will be lies in top of middle tree. And it take from us about `O(logN)` operations. Cool, doesn't it?

Actually we can calculate `any function`, that has 'neutral' element like `0` for sum or `1` for multiplication for empty nodes, and have `associative property` like $(2+3)+4 = 2 + (3+4) = 9$ for correct work of merge and split.

For example we can use concatenation of strings. Neutral element is ''. And 'abc' + ('bcd' + 'cde') = ('abc' + 'bcd') + 'cde'.

# Segment tree

Using fast computation of some function in segments of array not limited by Catresian tree. In fact, the ideas of these mechanisms came from Segment tree.

The main idea of segment tree is precalculate function on some segment and then use concatenation of segments. So, at the bottom we have out array. All of it elements. For each pair of elements (unique elements, mean 1 and 2; 3 and 4 and etc.) we calculate some function that was describe above and right result in new nodes, that creates a new layer of tree (second one). Than we repeat this in next layer and till the root, where we will have the result of function on the whole array. The deep of such tree will be exactly `logN`, not 'near'. Plus to this, we can not only precalculate some function, but even change the segments of array with logN speed (using lazy evaluation).

Important - it is not associated array. It is special container that help work with array to change it and quickly respond to different requests in segments of the array (such as maximum element). So, there are not `insert` or `delete` elements. Instead we have:

1. Construction - `O(N)`
2. Calc function on segment (a,b) - calculate some function (that precalculate by the tree) on segment - `O(logN)`
3. Add k to each value in segment (a,b) - change the value of array elements inside segment by k (+ or -). `O(logN)`

![segmentTree](images/SegmentTree.jpeg)

## Construction

Ok, just like the heap we will contain all elements inside usual array. 

Firstly, because our tree is binary and should be calculated with 'constant' numbers of elements in every layer we fill our array to nearest power of 2 (2\*\*k). So, now our array has a size 2\*\*k. But we need the space for other tree nodes. For this lets resize array to 2\*\*(k+1), because for all other nodes it is enough N elements, just believe.

Now lets fill our tree. We can make it recursively or with cycle. The more correct realization is cycle, but for visibility we will implement it recursively. Ok, as we know, each node should be calculate from two nodes at the bottom layer. In array it look like $a[i] = a[2*i] + a[2*i+1]$. Important - in such realization array are beginning from `1`, not `0`. So, we have a recursive formula, that we can implement with recursive function. But you will implement it at home).

The other way of implementation of such tree is create a list, that contain links to their children and parents. It more simple way but it may require 2-3 times more memory.

## Calc function on segment

Lets image that we need take the result of some function F on segment(a,b). To solve this we need look carefully at the number of situation that we can meet in process. First of all each segment have a borders - left one and right one. Call them `l` and `r`. So, the four situations:

1. `b` is less than `l` or `a` is bigger than `r` - in this situation we can just return neutral element ('0' for example)
2. `a` is less than `l` and `b` is bigger than `r` (mean that (l,r) lies inside (a,b)) - in this case we return the result of function that contains inside node because all elements of (l,r) are inside (a,b).
3. `a` is bigger than `l` and `b` is less than `r` (mean (a,b) lies inside (l,r)) - in this case we need to divide (l,r) into two parts (left and right) and recursively calculate in each branch. Then we calc function on two results that we will get from them. Due to the fact that the segment (l, r) will decrease in size in every step of recursion by 2 at some point we will meet one of the cases `1` or `2` and return answer.
4. `l` inside (a,b) and `r` is bigger than `b` or vice versa (like a,l,b,r or l,a,r,b - In this cases we need to make the same operation as above - divide (l,r) into two parts and calc inside them. Actually when we divide (l,r) into two parts we just calculate the function on a shorter segment because our current segment have the elements outside the required segment.

![segmentTreeSearch](images/segmentTreeSearch.png)

In picture above you can see the process of calculation of function on segment `(2,12)`. Segment (1,2) we divided into (1) and (2) and took (2). Segment (3,4), (5,8), (9,10) we took wholly. And segment (11,12) we divided into (11) and (12) and took (11). Make a note that segments (1,8), (1,4) and others we divide into two parts too.

## Change each element inside segment.

Ok, now final magic. Sometimes we may need to change values of element inside some segment of array. If we will do it the classic way by changing all elements one by one it takes `O(k)` steps. If k small enough than we can use it, but most often k~N. That means that complexity of this function  will be near `O(N)` which is pretty slow. But we have the tree with fast calculation of some function. Let's take this opportunity.

Change or add some number to an element is the function that fulfill for functions the rules above. But there is one important moment - we need to change each element inside segment, not only node that represent the segment. But it is too long. So, lets think a little - when we actually need changed elements? When we apply to them. And why we need change them now when we can change them at the moment of appeal. Right - no need. So lets change them right at the moment of appeal.

 However in this case we need some mechanism to remember changes in segment. But we already have a node that represent our segment. Lets then write a changes inside this node. And when we meet it in the process of calculating the value of the function we apply changes for function result and save how much required to change the data in the segment in the children's elements. 

 ![segmentTreeLazy](images/segmentTreeLazy.png)
 
 [segmentTreeLazy2](images/segmentTreeLazy2.png)

 So the final moment is to understand how to change function value if you know how much values of elements have 'changed'. Unfortunately you have to answer to this question by yourself because for every function there are individual scheme of work. For example for sum function we need add the change multiply by number of elements in segment (we can save it inside each node). And for maximum function (`RMQ task`) we need just add to previous max the value of the change.

To sum up we now know about new effective container - Segment tree that can help us to fast (O(logN)) calculate function on segments of array and plus to this give us opportunity to fast changes of values inside segments.